In [1]:
import io
import os
import sqlite3
import zipfile
from pathlib import Path

import pandas as pd
import regex
from tqdm import tqdm

In [2]:
DATA_DIR = Path('..') / 'data'
RAW_DIR = DATA_DIR / '00_raw'
INTERIM_DIR = DATA_DIR / '01_interim'

ZIP = RAW_DIR / 'idigbio_all_2020-03-30.zip'
DB = INTERIM_DIR / 'gazetteer_10_idigbio_2020-03-30.db'
CSV = INTERIM_DIR / 'idigbio_2020-03-30c.csv'

CHUNK = 1_000_000

In [3]:
if DB.exists():
    os.remove(DB)

## Columns to use for output

In [4]:
DWC_FIELDS = {f'dwc:{f}': f for f in """
    continent
    coordinatePrecision
    coordinateUncertaintyInMeters
    country
    countryCode
    county
    decimalLatitude
    decimalLongitude
    footprintSRS
    footprintSpatialFit
    footprintWKT
    georeferencedDate
    geodeticDatum
    georeferencedBy
    georeferenceProtocol
    georeferenceremarks
    georeferenceSources
    georeferenceVerificationStatus
    higherGeography
    higherGeographyID
    island
    islandGroup
    locality
    locationID
    locationRemarks
    maximumDepthInMeters
    maximumDistanceAboveSurfaceInMeters
    maximumElevationInMeters
    minimumDepthInMeters
    minimumDistanceAboveSurfaceInMeters
    minimumElevationInMeters
    municipality
    pointRadiusSpatialFit
    stateProvince
    verbatimCoordinateSystem
    verbatimCoordinates
    verbatimDepth
    verbatimElevation
    verbatimLatitude
    verbatimLocality
    verbatimLongitude
    verbatimSRS
    waterBody
""".split()}


IDIGBIO_FIELDS = {f'idigbio:{f}': f for f in """
    geoPoint
    isoCountryCode
""".split()}

OTHER_FIELDS = {'coreid': 'coreid'}  # Need to link data

FIELDS = {**OTHER_FIELDS, **DWC_FIELDS, **IDIGBIO_FIELDS}

### Get the headers from the zip file CSV

In [5]:
def get_headers(zip_file):
    with zipfile.ZipFile(ZIP) as zippy:
        with zippy.open(zip_file) as in_file:
            headers = in_file.readline()
    return [h.decode().strip() for h in sorted(headers.split(b','))]

### Write to database

In [6]:
def insert(zip_file, usecols, columns):
    table = zip_file.split('.')[0]

    with sqlite3.connect(DB) as cxn:
        with zipfile.ZipFile(ZIP) as zippy:
            with zippy.open(zip_file) as in_file:

                reader = pd.read_csv(
                    in_file, dtype=str, keep_default_na=False,
                    chunksize=CHUNK, usecols=usecols)

                if_exists = 'replace'

                for df in tqdm(reader):
                    df = df.rename(columns=columns)

                    df.to_sql(table, cxn,
                              if_exists=if_exists, index=False)

                    if_exists = 'append'

In [7]:
def wrapper(zip_file):
    headers = get_headers(zip_file)

    usecols = [h for h in headers if h in FIELDS]
    columns = {h: FIELDS[h] for h in usecols}

    insert(zip_file, usecols, columns)

## Write to DB

In [8]:
wrapper('occurrence.csv')

122it [27:26, 13.50s/it]


In [9]:
wrapper('occurrence_raw.csv')

122it [42:06, 20.71s/it]


##  Update database

In [11]:
sql = """
    create index if not exists occ_coreid on occurrence (coreid);
    create index if not exists raw_coreid on occurrence_raw (coreid);
"""

with sqlite3.connect(DB) as cxn:
    cxn.executescript(sql)

In [14]:
sql = """
    alter table occurrence rename column isoCountryCode to idigbio_countrycode;
    alter table occurrence add column idigbio_decimallatitude_wgs84;
    alter table occurrence add column idigbio_decimallongitude_wgs84;
"""

with sqlite3.connect(DB) as cxn:
    cxn.executescript(sql)

In [17]:
sql = """
    update occurrence
       set idigbio_decimallatitude_wgs84  = json_extract(geoPoint, '$.lat'),
           idigbio_decimallongitude_wgs84 = json_extract(geoPoint, '$.lon')
     where geoPoint is not null
       and geoPoint <> '';
"""

with sqlite3.connect(DB) as cxn:
    cxn.execute(sql)
    cxn.commit()

## Write output to CSV file

In [18]:
if CSV.exists():
    os.remove(CSV)

In [19]:
sql = """
    select occurrence_raw.*,
           idigbio_countrycode,
           idigbio_decimallatitude_wgs84,
           idigbio_decimallongitude_wgs84
    from occurrence_raw
    left join occurrence using (coreid);
    """

In [20]:
with open(CSV, 'a') as out:
    with sqlite3.connect(DB) as cxn:

        reader = pd.read_sql(sql, cxn, chunksize=CHUNK)

        header = True

        for df in tqdm(reader):
            df.to_csv(out, header=header, index=False)

            header = False

122it [53:14, 26.19s/it]


In [21]:
!head -1 "$CSV"

coreid,continent,coordinatePrecision,coordinateUncertaintyInMeters,country,countryCode,county,decimalLatitude,decimalLongitude,footprintSRS,footprintSpatialFit,footprintWKT,geodeticDatum,georeferenceProtocol,georeferenceSources,georeferenceVerificationStatus,georeferencedBy,georeferencedDate,higherGeography,higherGeographyID,island,islandGroup,locality,locationID,locationRemarks,maximumDepthInMeters,maximumElevationInMeters,minimumDepthInMeters,minimumElevationInMeters,municipality,pointRadiusSpatialFit,stateProvince,verbatimCoordinateSystem,verbatimCoordinates,verbatimDepth,verbatimElevation,verbatimLatitude,verbatimLocality,verbatimLongitude,verbatimSRS,waterBody,idigbio_countrycode,idigbio_decimallatitude_wgs84,idigbio_decimallongitude_wgs84


In [22]:
!gzip "$CSV"